# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,9.71,95,2,1.82,ZA,1723061926
1,1,bogorodskoye,56.5500,37.8833,14.22,93,89,1.39,RU,1723062085
2,2,edinburgh of the seven seas,-37.0676,-12.3116,8.10,64,99,14.19,SH,1723061924
3,3,jamestown,42.0970,-79.2353,25.31,63,100,4.63,US,1723061920
4,4,lihue,21.9789,-159.3672,28.58,69,75,6.17,US,1723062089


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      size = "Humidity",
                                      tiles = "OSM",
                                      frame_width = 800,
                                      frame_height = 600

                                     )


# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,al bayda',32.7627,21.7551,22.14,80,0,2.78,LY,1723062143
95,95,south lockport,43.1501,-78.6967,26.59,54,1,2.68,US,1723062217
188,188,shimoda,34.6667,138.9500,25.61,85,0,0.95,JP,1723062343
193,193,uzgen,40.7699,73.3007,23.79,32,0,3.03,KG,1723062349
214,214,presidente olegario,-18.4178,-46.4181,25.22,30,0,3.30,BR,1723062378
220,220,ypsonas,34.6884,32.9574,26.54,83,0,4.12,CY,1723062386
249,249,mitsamiouli,-11.3847,43.2844,25.29,72,1,3.53,KM,1723062426
263,263,piacabucu,-10.4056,-36.4344,24.14,75,0,2.63,BR,1723062446
266,266,fayzabad,37.1166,70.5800,21.72,30,0,0.62,AF,1723062450
281,281,montmagny,48.9734,2.3469,21.36,54,0,3.60,FR,1723062469


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,al bayda',LY,32.7627,21.7551,80,
95,south lockport,US,43.1501,-78.6967,54,
188,shimoda,JP,34.6667,138.9500,85,
193,uzgen,KG,40.7699,73.3007,32,
214,presidente olegario,BR,-18.4178,-46.4181,30,
220,ypsonas,CY,34.6884,32.9574,83,
249,mitsamiouli,KM,-11.3847,43.2844,72,
263,piacabucu,BR,-10.4056,-36.4344,75,
266,fayzabad,AF,37.1166,70.5800,30,
281,montmagny,FR,48.9734,2.3469,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al bayda' - nearest hotel: فندق الهديل
south lockport - nearest hotel: Quality Inn
shimoda - nearest hotel: ホテルウラガ
uzgen - nearest hotel: Кунрат
presidente olegario - nearest hotel: No hotel found
ypsonas - nearest hotel: Club Aphrodite
mitsamiouli - nearest hotel: Hotel le Paradis
piacabucu - nearest hotel: O Leão
fayzabad - nearest hotel: Kokcha Hotel
montmagny - nearest hotel: Entreprises val de france
morondava - nearest hotel: Hotel Sharon Menabe
piui - nearest hotel: Rib's Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
42,al bayda',LY,32.7627,21.7551,80,فندق الهديل
95,south lockport,US,43.1501,-78.6967,54,Quality Inn
188,shimoda,JP,34.6667,138.9500,85,ホテルウラガ
193,uzgen,KG,40.7699,73.3007,32,Кунрат
214,presidente olegario,BR,-18.4178,-46.4181,30,No hotel found
220,ypsonas,CY,34.6884,32.9574,83,Club Aphrodite
249,mitsamiouli,KM,-11.3847,43.2844,72,Hotel le Paradis
263,piacabucu,BR,-10.4056,-36.4344,75,O Leão
266,fayzabad,AF,37.1166,70.5800,30,Kokcha Hotel
281,montmagny,FR,48.9734,2.3469,54,Entreprises val de france


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   size = "Humidity",
                                   tiles = "OSM",
                                   frame_width = 800,
                                   frame_height = 600,
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)